In [ ]:
import pandas as pd
import numpy as np
import dash
from datetime import date
from dash import dcc
from dash import html
import shapely
from dash.dependencies import Input, Output
from shapely.geometry import Point, LineString, Polygon
import plotly.express as px
from dash.exceptions import PreventUpdate
import base64

import geojson

encoded_image = base64.b64encode(open('/home/pfernandes/Downloads/cmp.png', 'rb').read()).decode('ascii')


with open("/home/pfernandes/Downloads/porto_tertiary_pedestrian.geojson", encoding="utf-8") as file:
    data_tertiary_pedestrian = geojson.load(file)

with open("/home/pfernandes/Downloads/porto_railway_subway_streams.geojson", encoding="utf-8") as file:
    data_railway_subway_streams = geojson.load(file)

with open("/home/pfernandes/Downloads/porto_highway_system.geojson", encoding="utf-8") as file:
    data_highway_system = geojson.load(file)

roads = {}
rails = {}
waterways = {}

for path in data_highway_system["features"]:
    # street data
    if "highway" in path["properties"].keys():
        if path["properties"]["highway"] not in roads.keys():
            roads[path["properties"]["highway"]] = []

        if path["geometry"]["type"] == "Polygon":
            cart = [[x, y] for x, y in path["geometry"]["coordinates"][0]]
            roads[path["properties"]["highway"]].append(Polygon(cart))



        elif path["geometry"]["type"] == "LineString":
            cart = [[x, y] for x, y in path["geometry"]["coordinates"]]
            roads[path["properties"]["highway"]].append(LineString(cart))

for path in data_tertiary_pedestrian["features"]:
    # street data
    if "highway" in path["properties"].keys():
        if path["properties"]["highway"] not in roads.keys():
            roads[path["properties"]["highway"]] = []

        if path["geometry"]["type"] == "Polygon":
            cart = [[x, y] for x, y in path["geometry"]["coordinates"][0]]
            roads[path["properties"]["highway"]].append(Polygon(cart))


        elif path["geometry"]["type"] == "LineString":
            cart = [[x, y] for x, y in path["geometry"]["coordinates"]]
            roads[path["properties"]["highway"]].append(LineString(cart))

for path in data_railway_subway_streams["features"]:
    # street data
    if "waterway" in path["properties"].keys():
        if path["properties"]["waterway"] not in waterways.keys():
            waterways[path["properties"]["waterway"]] = []

        if path["geometry"]["type"] == "Polygon":
            cart = [[x, y] for x, y in path["geometry"]["coordinates"][0]]
            waterways[path["properties"]["waterway"]].append(Polygon(cart))


        elif path["geometry"]["type"] == "LineString":
            cart = [[x, y] for x, y in path["geometry"]["coordinates"]]
            waterways[path["properties"]["waterway"]].append(LineString(cart))

    # railway data
    elif "railway" in path["properties"].keys():
        if path["properties"]["railway"] not in rails.keys():
            rails[path["properties"]["railway"]] = []

        if path["geometry"]["type"] == "Polygon":
            cart = [[x, y] for x, y in path["geometry"]["coordinates"][0]]
            rails[path["properties"]["railway"]].append(Polygon(cart))

        elif path["geometry"]["type"] == "LineString":
            cart = [[x, y] for x, y in path["geometry"]["coordinates"]]
            rails[path["properties"]["railway"]].append(LineString(cart))

values = {"roads": roads, "rails":  rails, "waterways": waterways}
options_checklist = list(values.keys())
app = dash.Dash(external_stylesheets=[
                    'https://codepen.io/chriddyp/pen/bWLwgP.css'])

app.layout = html.Div([
    html.Div([html.Img(src='data:image/png;base64,{}'.format(encoded_image),
             style={'float': 'left', 'padding': '0', 'margin': '0px', 'height': '80px'
                    })],className='row',  style={'marginBottom': '0px', 'backgroundColor': '#ffffff', 'padding': '0'}),
    html.H1('City of Porto Mobility Dashboard', style={'fontWeight': 'bold',
                                                                 'fontSize': '30px',
                                                                 'marginTop': '30px',
                                                                 'marginLeft': '600px',
                                                                    'marginBottom':"20px"}),
    html.Div([dcc.Checklist(id='way-type',
                  options=[{'label': str.capitalize(k), 'value': k} for k in options_checklist],
                  value=[],
                  labelStyle={'display': 'inline'}
                  ),
    dcc.Dropdown(options=[], multi=True, id='dropdown', style={'width':'30%', 'display':'block',
                                                            'backgroundColor':'#ffffff',
                                                            'borderStyle':'solid',
                                                            'borderWidth':'1px',
                                                            'borderRadius':'5px'})]),
    html.Div(id='dd-output-container'),


    html.Div(id="graph-container")

])

@app.callback(
    [Output("dropdown", "options")],
     # Output("big-dict", "style")],
    Input("way-type", 'value'),

)
def update_dropdown_options(checklist):
    if not checklist:
        raise PreventUpdate
    dropdown_vals = []
    for item in checklist:
        dropdown_vals.append(values[item])

    return [[ {"label":k , "value":k } for vals in dropdown_vals for k in list(vals.keys())]]

@app.callback(
    [Output("graph-container", "children")],
    [Input("dropdown", "value"),
     Input("way-type", 'value')]
)
def create_graph(dropdown, way_type):
    if not way_type or not dropdown:
        raise PreventUpdate
    lats = []
    lons = []
    names = []
    i = 0
    for feature in values[way_type[0]][dropdown[0]]:

        if isinstance(feature, shapely.geometry.linestring.LineString):
            linestrings = [feature]
        elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
            linestrings = feature.geoms
        else:
            continue
        for linestring in linestrings:
            x, y = linestring.xy
            lats = np.append(lats, y)
            lons = np.append(lons, x)
            # names = np.append(names, i*len(y))
            lats = np.append(lats, None)
            lons = np.append(lons, None)
            # names = np.append(names, None)

    fig = px.line_mapbox(lat=lats, lon=lons,
                         mapbox_style="stamen-terrain", zoom=1)

    fig.update_layout(
        margin={'l': 0, 't': 0, 'b': 0, 'r': 0},
        mapbox={
            'center': {'lon': -8.6, 'lat': 41.15},
            'style': "stamen-terrain",
            'zoom': 10})
    return [dcc.Graph(figure = fig)]



if __name__ == '__main__':

    app.run_server(debug=True)


